# โหลดข้อมูล Testing

In [546]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from  sklearn.feature_extraction.text import TfidfVectorizer

Titanic_data = 'Titanic/test.csv' # เก็บชื่อที่อยู่ไฟล์
df = pd.read_csv(Titanic_data) # โหลดไฟล์
df.head(100) # preview
df.loc[304] 

PassengerId                                 1196
Pclass                                         3
Name           McCarthy, Miss. Catherine Katie""
Sex                                       female
Age                                          NaN
SibSp                                          0
Parch                                          0
Ticket                                    383123
Fare                                        7.75
Cabin                                        NaN
Embarked                                       Q
Name: 304, dtype: object

In [547]:
df.head()
#df.shape # ดูขนาดข้อมูล

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [548]:
list_Name = ['Klaew, Mr. Chaiwat','Klaew, Mr. Wanchai','Klaew, Mr. Chaiwat','Klaew, Mr. Wanchai','Klaew, Mr. Chaiwat','Klaew, Mr. Wanchai']
datas = list(list_Name)
#H = pd.DataFrame(datas,columns=['Name'])
X_test = df.Name # เลือกเฉพาะชื่อมาทำ features
#X_test = H.Name.loc[:5]

# Data Preprocessing
- 1. ตัดคำแยกเป็น : LastName, Prefix. name
- 2. แยก column เป็น : prefix_name, last_name, name
- 3. นับจำนวน : word_lastName, syllable_lastName, word_name,syllable_name
- 4. label encoding : prefix_name_, word_lastName_, syllable_lastname_, word_name_, syllable_name_
- 5. Normalizationให้ค่ามีscaleระหว่าง 0-1 : prefix_name_, word_last_name_, syllable_lastname_, word_name_, syllable_name_

# Data pipeline
- 1.Data Preprocessing
- 2.model_RFC predict X_test
- 3.Display

# Data Preprocessing(Code)

In [549]:
# ข้อมูล input
X = X_test
len(X)

418

# โค้ด python function สำหรับนับพยางค์

In [550]:
import re
def syllables(word) :
    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

In [551]:
#ทำ labelencoding สำหรับ X (input)
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

def labelEncode(data, columns):
    for i in columns:
#         print(i)
        lb = LabelEncoder().fit_transform(data[i])
        data[i + '_'] = lb

# 1.สำหรับ Preprocessing

In [ ]:
# import library
from nltk.tokenize import WhitespaceTokenizer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler as Scaler # Normalization 0-1 เพราะ Variance มีค่าต่างกันแต่ละตัวแปร

def preprocessing(X):
    
    ### 1. ตัดคำแยกเป็น : LastName, Prefix. name ###
    
    tokenized_lists = [] # สร้าง list ไว้รอเก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง
    for i in range(len(X)):
        tokenized_lists.append((WhitespaceTokenizer(), i)) # เก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง(ไว้ใช้ต่อไป)    
    results = []
    for tokenized, i in tokenized_lists:
        tokenized_list = tokenized.tokenize(X[i]) 
        results.append(tokenized_list) # เอาผลไว้ใน results
        
    ### 2. แยก column เป็น : last_name,prefix_name, name ###
    
    # เก็บข้อมูลลง DataFrame
    df_text = pd.DataFrame(results)  # ยังไม่ต้องระบุชื่อคอลัมน์
    # การตัดคำจะถูกแยกออกเป็นตัดแล้วได้ 1.ได้คำที่คอลัมน์ 1 พอดี 2. ไปอยู่ที่คอลัมน์ 2 แก้โดยหา idx_fail แล้วเปลี่ยน ณ index นั้นๆ
    # กำจัด missing data ด้วยการแทน str : ' '
    for i in range(len(df_text.columns)):
        df_text[i] = df_text[i].fillna(' ')
        
        
    # กรณีถ้า คอลัมน์ไม่ถึง 4 ให้สร้าง index มาเติมให้ครบ 4
    df_text.head()
    x = len(df_text.columns) # จำนวนคอลัมน์ df ที่มี
    final = 5
    for x in range(x,4):
        df_text[x] = ''
        
    Prefix_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.', 'Dona.']
    Prefix_special = ''
    
    idx_fail = df_text[1].loc[
        
                          (df_text[1] != Prefix_list[0]) & (df_text[1]!= Prefix_list[1]) & (df_text[1] !=Prefix_list[2]) &  (df_text[1] !=Prefix_list[3]) &
                          (df_text[1] != Prefix_list[4]) & (df_text[1]!= Prefix_list[5]) & (df_text[1] !=Prefix_list[6]) &  (df_text[1] !=Prefix_list[7]) &
                          (df_text[1] != Prefix_list[8]) & (df_text[1]!= Prefix_list[9]) & (df_text[1] !=Prefix_list[10]) &  (df_text[1] !=Prefix_list[11]) &
                          (df_text[1] != Prefix_list[12]) & (df_text[1]!= Prefix_list[13]) & (df_text[1] !=Prefix_list[14]) & (df_text[1] !=Prefix_list[15])
        
                          ] 
            
    for i in idx_fail.index:
         for j in Prefix_list:
                # ตรวจดูข้อมูลแล้วเปลี่ยนให้ถูกต้องตามแต่กรณี
                if (df_text[2].loc[i] == j):
                    df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i] # รวมคอลัมน์ 0, 1
                    df_text[1].loc[i] = j                 # แทนที่คอลัมน์ 1 ด้วยคอลัมน์ 2
                    
                elif df_text[2].loc[i] == 'Countess.':
                    df_text[1].loc[i] = 'the Countess.' # แทนที่ the ด้วย 'the Countess.'
                    
                elif df_text[3].loc[i] == j:
                    df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i]+' '+ df_text[2].loc[i] # รวมคอลัมน์ 0, 1, 2
                    df_text[1].loc[i] = j          # แทนที่คอลัมน์ 2 ด้วยคอลัมน์ 3
                    
    for i in df_text.index:
        
          for j in Prefix_list:
                

            # กรณีคอลัมน์ 1 = Prefix_list , คอลัมน์ 2 = Prefix_list
                if (df_text[1].loc[i]==j) & (df_text[2].loc[i]==j):
                    df_text[2].loc[i] = ''
                    k = 2
                    for k in range(len(df_text.loc[i])-3):
                        df_text[2].loc[i] += df_text[k+3].loc[i] + ' '
            
            # กรณีคอลัมน์ 1 = Prefix_list , คอลัมน์ 3 = Prefix_list
                elif (df_text[1].loc[i]==j) & (df_text[2].loc[i]!=j) &(df_text[3].loc[i]==j):
                    df_text[2].loc[i] = ''
                    df_text[3].loc[i] = ''
                    k = 2
                    for k in range(len(df_text.loc[i])-2):
                        df_text[2].loc[i] += df_text[k+2].loc[i] + ' '
             # ทั่วไป กรณีคอลัมน์ 1 = Prefix_list  
                elif (df_text[1].loc[i]==j) & (df_text[2].loc[i]!=j) &(df_text[3].loc[i]!=j):
                    k = 2
                    for k in range(len(df_text.loc[i])-3):
                        df_text[2].loc[i] += ' ' + df_text[k+3].loc[i]
          
    # กรณีคำนำหน้าเป็น 'the Countess.'   
    if (df_text[1].loc[i]== 'the Countess.'):
        
        k = 2
        df_text[2].loc[i] = ' '
        for k in range(len(df_text.loc[i])-2):
            df_text[2].loc[i] += ' ' + df_text[k+2].loc[i]    
     
    # เก็บไว้ใน DataFrame พร้อมตั้งชื่อคอลัมน์
    column_features = ['last_name', 'prefix_name', 'name']
    i = 0    
    for feature in column_features: 
        df_text[feature] = df_text[i]
        i += 1
        
    # เลือก features ที่จะนำมาใช้
    cols = ['last_name', 'prefix_name', 'name']
    X_features = df_text[cols]
    
    # นับจำนวน : word_lastName, syllable_lastName, word_name,syllable_name                            
    tfv = TfidfVectorizer() # แบ่งคำ
    count_word_LastName = []
    # สำหรับนับจำนวนคำ word_lastName
    for i in X_features.index:
        
        text = X_features.last_name[i]
        text_list = [text]
        words = tfv.fit_transform(text_list)
        text_cut = tfv.get_feature_names()
        count_word_LastName.append(len(text_cut))
        
    # สำหรับนับจำนวนพยางค์ syllable_Lastname     
    X_features['count_word_LastName'] = count_word_LastName    
    count_syllable_LastName = []
    for i in X_features.index:
        count_syllable_LastName.append(syllables(X_features.last_name[i]))

    X_features['count_syllable_LastName'] = np.array(count_syllable_LastName)

    # สำหรับนับจำนวนคำ word_name
    count_word_Name = []
    for i in X_features.index:
        text = X_features.name[i]
        text_list = [text]
        words = tfv.fit_transform(text_list)
        text_cut = tfv.get_feature_names()
        count_word_Name.append(len(text_cut)) 
    X_features['count_word_Name'] = np.array(count_word_Name)    
    
    # สำหรับนับจำนวนพยางค์ syllable_name
    count_syllable_Name = []
    for i in X_features.index:
        count_syllable_Name.append(syllables(X_features.name[i]))
       
    X_features['count_syllable_Name'] = np.array(count_syllable_Name) 
      
    ### 4.label encoding : prefix_name_, word_lastName_, syllable_lastname_, word_name_, syllable_name_ ###
    columns =  ['prefix_name', 'count_word_LastName', 'count_syllable_LastName', 'count_word_Name', 'count_syllable_Name']
    labelEncode(X_features, columns)
        
            
    ### 5.Normalizationให้ค่ามีscaleระหว่าง 0-1 : prefix_name_, word_last_name_, syllable_lastname_, word_name_, syllable_name_ ###       
    # เลือก feature มาทำ normalization
    col_sel = ['prefix_name_', 'count_word_LastName_', 'count_syllable_LastName_', 'count_word_Name_', 'count_syllable_Name_']
    X_pp = X_features[col_sel]
    X = X_pp 
    sc = Scaler()
    X_sc = sc.fit_transform(X) # Normalization 0-1
    df_sc = pd.DataFrame(X_sc, columns=X.columns) # เก็บ ที่ทำtransform แล้วลงในตัวแปร df_sc    
        

        
 
    
    
    return  df_sc

# สร้าง Pipeline

In [553]:
df_text = preprocessing(X)
df_text.sample(5)

,prefix_name_,count_word_LastName_,count_syllable_LastName_,count_word_Name_,count_syllable_Name_
159,0.750,0.0,0.166667,0.4,0.428571
362,0.750,0.0,0.166667,0.8,0.571429
12,0.750,0.0,0.166667,0.6,0.571429
139,0.625,0.0,0.166667,0.2,0.285714
166,0.625,0.0,0.166667,0.0,0.071429


In [554]:
df_text = preprocessing(X)
df_text

,prefix_name_,count_word_LastName_,count_syllable_LastName_,count_word_Name_,count_syllable_Name_
0,0.625,0.0,0.166667,0.0,0.071429
1,0.750,0.0,0.166667,0.4,0.285714
2,0.625,0.0,0.166667,0.2,0.214286
3,0.625,0.0,0.000000,0.0,0.071429
4,0.750,0.0,0.333333,0.4,0.571429
...,...,...,...,...,...
413,0.625,0.0,0.166667,0.0,0.000000
414,0.125,1.0,1.000000,0.0,0.142857
415,0.625,0.0,0.166667,0.2,0.285714
416,0.625,0.0,0.166667,0.0,0.142857


In [555]:
df_text = preprocessing(X)

In [556]:
len(X)

418

In [557]:
df_text.loc[414]

prefix_name_                0.125000
count_word_LastName_        1.000000
count_syllable_LastName_    1.000000
count_word_Name_            0.000000
count_syllable_Name_        0.142857
Name: 414, dtype: float64

In [558]:
Prefix_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.']
len(Prefix_list)

16

In [559]:
df_text.sample(10) # ที่ index 414

,prefix_name_,count_word_LastName_,count_syllable_LastName_,count_word_Name_,count_syllable_Name_
254,0.625,0.0,0.166667,0.0,0.071429
41,0.625,0.0,0.166667,0.2,0.214286
7,0.625,0.0,0.166667,0.2,0.214286
282,0.500,0.0,0.166667,0.0,0.142857
267,0.625,0.0,0.166667,0.2,0.214286
295,0.625,0.0,0.333333,0.0,0.071429
35,0.625,0.0,0.500000,0.2,0.642857
116,0.625,0.0,0.166667,0.0,0.000000
84,0.625,0.0,0.000000,0.2,0.142857
39,0.625,0.0,0.000000,0.0,0.000000


In [560]:
#df_text.loc[414] 

In [561]:
 # สำหรับนับพยางค์ syllable_lastName
        count_syllable_lastName.append(syllables(X_features.last_name[i]))
        X_features['count_syllable_lastname'] = np.array(count_syllable_lastname)
        
        # สำหรับนับจำนวนคำ word_name
        text = X_features.name[i]
        text_list = [text]
        words = tfv.fit_transform(text_list)
        text_cut = tfv.get_feature_names()
        count_word_lastname.append(len(text_cut)) 
        X_features['count_word_name'] = [count_word_name]

        # สำหรับนับพยางค์ syllable_lastName
        count_syllable_name.append(syllables(X_features.name[i]))
        X_features['count_syllable_name'] = [count_syllable_name]
        

IndentationError: unexpected indent (<ipython-input-561-2d01d8e25906>, line 2)

In [ ]:
H.dtypes